In [1]:
# LIBRERIAS
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict, List
from IPython.display import display, Markdown

In [2]:
# CARGAR VARIABLES DE ENTORNO APIS
load_dotenv(override=True)

True

In [3]:
# DEFINIR AGENTE DE BUSQUEDA

INSTRUCTIONS = "Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y \
producí una descripción concisa de los resultados. La descripción debe tener 2-3 párrafos y menos de 300 \
palabras. Captura los puntos principales. Escribe de manera concisa, no es necesario tener frases completas o buena \
gramática. Esto será consumido por alguien que está sintetizando un informe, por lo que es vital que captures el \
esencia y ignores cualquier fluff. No incluyas ningún comentario adicional más que la descripción en sí."

search_agent = Agent(
    name="Agente de búsqueda",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")], # RESPUESTAS CONCISAS
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # INDICAS AL AGENTE QUE DEBE BUSCAR POR INTERNET ANTES DE RESPONDER
)

In [4]:
# PONER APRUEBA AGENTE DE BUSQUEDA

message = "Principales commodities a invertir mediante ETFs para 2026"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

Para 2026, se destacan varios ETFs de commodities que podrían ser atractivos para los inversores:

**Oro y Plata**: Ambos metales preciosos han mostrado un rendimiento excepcional en 2025, con el oro superando los 4.500 dólares por onza y la plata alcanzando los 80 dólares. Se espera que esta tendencia continúe en 2026, aunque a un ritmo más moderado. Expertos como JP Morgan y Société Générale prevén que el oro podría llegar a los 5.000 dólares, impulsado por la demanda de bancos centrales y su estatus como refugio en tiempos de incertidumbre económica. Para invertir en oro, el SPDR Gold Shares (GLD) es una opción destacada, mientras que para la plata, el iShares Silver Trust (SLV) es una alternativa popular. ([cincodias.elpais.com](https://cincodias.elpais.com/mercados-financieros/2026-01-04/que-esperar-del-oro-y-la-plata-en-2026-tras-un-ano-de-maximos-historicos.html?utm_source=openai))

**Uranio**: Con el creciente enfoque en la energía nuclear y la transición energética, el uranio ha ganado relevancia. El Global X Uranium ETF (URA) ofrece exposición a empresas involucradas en la minería y producción de uranio, beneficiándose de la creciente demanda energética. ([rankia.pe](https://www.rankia.pe/blog/gestion-indexada/7167915-mejores-etfs-invertir-peru-febrero-2026?utm_source=openai))

**Semiconductores**: La demanda de semiconductores, impulsada por la inteligencia artificial y la computación en la nube, sigue en aumento. El VanEck Vectors Semiconductor ETF (SMH) proporciona exposición a empresas líderes en este sector, como Nvidia y TSMC. ([rankia.com.ar](https://www.rankia.com.ar/blog/fondos-indexados-argentina/7108322-etfs-prometedores-para-invertir-2026?utm_source=openai))

Al considerar inversiones en commodities a través de ETFs, es esencial evaluar las tendencias del mercado, la demanda futura y los riesgos asociados para tomar decisiones informadas. 

In [5]:
# DEFINIR EL AGENTE PLANIFICADOR PARA CENTRAR LA BUSQUEDA

HOW_MANY_SEARCHES = 3 # Número de busquedas

INSTRUCTIONS = f"Eres un asistente de investigación útil. Dado un término de búsqueda, \
produce un conjunto de búsquedas web para realizar para responder la consulta. \
Salida: {HOW_MANY_SEARCHES} términos para consultar."

# Define salidas estructuradas mediante Pydantic

class WebSearchItem(BaseModel):
    reason: str = Field(description="Tu razonamiento de por qué esta búsqueda es importante para la consulta.")

    query: str = Field(description="El término de búsqueda para usar para la búsqueda web.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="Una lista de búsquedas web a realizar para responder la consulta.")

planner_agent = Agent(
    name="Agente de planificación",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
# PONER APRUEBA AGENTE PLANIFICADOR

message = "Principales commodities a invertir mediante ETFs para 2026"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Identificar los commodities más prometedores para invertir en el contexto económico actual y futuro.', query='principales commodities en los que invertir 2026'), WebSearchItem(reason='Conocer los ETFs más populares y rentables relacionados con commodities.', query='mejores ETFs commodities 2026'), WebSearchItem(reason='Evaluar tendencias y pronósticos de mercado para determinar oportunidades de inversión en commodities.', query='tendencias commodities inversión 2026')]


In [7]:
# DEFINIR HERRAMIENTA PARA ENVIAR EMAIL CON LA BUSQUEDA

@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto y el cuerpo HTML proporcionados """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("fmarti82sibilattec@gmail.com") # Cambiar a tu correo electrónico verificado
    to_email = To("fmarti82sibilattec@gmail.com") # Cambiar a tu correo electrónico
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
class FinanceGuardrailOutput(BaseModel):
    tripwire_triggered: bool
    issues: List[str]

finance_guardrail_agent = Agent(
    name="Guardrail financiero (frase obligatoria)",
    instructions="""
    Comprueba si el texto contiene esta frase:
    Este contenido no son consejos de inversión, sino contenido divulgativo.
    - Si la contiene: tripwire_triggered=false, issues=[], rewrite_instructions=""
    - Si NO la contiene: tripwire_triggered=true, issues=["Falta la frase obligatoria."],
    """
    .strip(),
    output_type=FinanceGuardrailOutput,
    model="gpt-4o-mini",
)

@input_guardrail
async def guardrail_against_investing_cta(ctx, agent, message: str):
    result = await Runner.run(finance_guardrail_agent, message, context=ctx.context)
    gr = result.final_output

    return GuardrailFunctionOutput(
        output_info={"finance_guardrail": gr.model_dump()},
        tripwire_triggered=bool(gr.tripwire_triggered),
    )

In [9]:
# DEFINIR EL AGENTE EMAIL

INSTRUCTIONS = """Eres capaz de enviar un correo electrónico HTML bien formateado basado en un informe detallado.
Se te proporcionará un informe detallado. Debes usar tu herramienta para enviar un correo electrónico, proporcionando el 
informe convertido en HTML limpio, bien presentado con un asunto adecuado.
"""

email_agent = Agent(
    name="Agente de correo electrónico",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_investing_cta]
)

In [10]:
# DEFINIR EL AGENTE QUE VA A ELABORAR UN INFORME SOBRE LAS BUSQUEDAS

INSTRUCTIONS = (
    "Eres un investigador senior encargado de escribir un informe coherente para una consulta de investigación. "
    "Se te proporcionará la consulta original y algunas investigaciones iniciales realizadas por un asistente de investigación.\n"
    "Primero, debes elaborar un esquema para el informe que describa la estructura y "
    "flujo del informe. Luego, genera el informe y devuelve ese como tu salida final.\n"
    "La salida final debe estar en formato markdown, y debe ser larga y detallada "
    "para 5-10 páginas de contenido, al menos 1000 palabras.\n\n"

    "OBLIGATORIO:\n"
    "- Mantén un tono divulgativo y analítico; no ofrezcas asesoramiento financiero personalizado.\n"
    "- No utilices lenguaje de recomendación directa al lector (por ejemplo: 'deberías invertir', 'te recomiendo').\n"
    "- Incluye AL FINAL del informe una sección titulada 'Aviso importante' que contenga EXACTAMENTE esta frase:\n"
    "  \"Este contenido no son consejos de inversión, sino contenido divulgativo.\""
)

class ReportData(BaseModel):
    short_summary: str = Field(description="Un resumen de 2-3 párrafos de los resultados.")

    markdown_report: str = Field(description="El informe final")

    follow_up_questions: list[str] = Field(description="Temas sugeridos para investigar más")

writer_agent = Agent(
    name="Agente de escritura",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [11]:
# FUNCIONES ASINCRONAS PARA PLANIFICAR Y EJECUTAR LAS BUSQUEDAS
# PLANIFICA BUSQUEDAS A REALIZAR
async def plan_searches(query: str):
    """ Utilice planner_agent para planificar qué búsquedas ejecutar para la consulta """
    print("Planificando búsquedas...")
    result = await Runner.run(planner_agent, f"Consulta: {query}")
    print(f"Se realizarán {len(result.final_output.searches)} búsquedas")
    return result.final_output

# EJECUTA VARIAS BUSQUEDAS A LA VEZ EN LUGAR DE UNA EN UNA
async def perform_searches(search_plan: WebSearchPlan):
    """ Llama a search() para cada elemento en el plan de búsqueda """
    print("Buscando...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Búsqueda finalizada")
    return results
    
# EJECUTA ESAS BUSQUEDAS
async def search(item: WebSearchItem):
    """ Usa el agente de búsqueda para ejecutar una búsqueda web para cada elemento en el plan de búsqueda """
    input = f"Término de búsqueda: {item.query}\nRazón para buscar: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [12]:
# ESCRIBE INFORME DE BUSQUEDAS Y ENVIA EMAIL
async def write_report(query: str, search_results: list[str]):
    """ Usa el agente de escritura para escribir un informe basado en los resultados de la búsqueda """
    print("Pensando sobre el informe...")
    input = f"Consulta original: {query}\nResultados de búsqueda resumidos: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Informe finalizado")
    return result.final_output

async def send_email(report: ReportData):
    """ Usa el agente de correo electrónico para enviar un correo electrónico con el informe """
    print("Escribiendo correo electrónico...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Correo electrónico enviado")
    return report

In [13]:
# PUESTA EN MARCHA DE TODAS LAS FUNCIONES

query = "Principales commodities a invertir mediante ETFs para 2026"

with trace("Investigación"):
    print("Iniciando investigación...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("¡Felicidades!")


Iniciando investigación...
Planificando búsquedas...
Se realizarán 3 búsquedas
Buscando...
Búsqueda finalizada
Pensando sobre el informe...
Informe finalizado
Escribiendo correo electrónico...
Correo electrónico enviado
¡Felicidades!
